# 3D Face Modeling (Basic)

There's multiple approach of 3D Face Modeling. In this notebook, i want to introduce 3D Morphable Model (3DMM).

## 1. 3D Morphable Model

[3DMM](https://cseweb.ucsd.edu/~ravir/6998/papers/p187-blanz.pdf) was first introduced in 1999 by **Volker Blanz** and **Thomas Vetter**. This method was later influential and became one of the most popular approach about 3D facial modeling.

### 1.1 Facial representation

We represent the geometry of a face with a shape vector $S=[X_1,Y_1,Z_1,X_2,Y_2,Z_2,...,X_n,Y_n,Z_n]^T\in \mathbb{R}^{3n}$ that contains the $x,y,z$ coordinates of n vertices, and a texture vector $[R_1,G_1,B_1,R_2,G_2,B_2,...,R_n,G_n,B_n]^T$ that represents $n$ corresponding R,G,B color value of each vertice. Each vertice is a point in the face, and we need the correspondence between faces in the dataset to satisfied this assumption. New shape $S_{\text{model}}$ and new texture $T_{\text{model}}$ can be expressed in barycentric coordinate as linear combination of shape and texture of $m$ exemplar faces in the dataset:

$$
S_{mod}=\sum_{i=1}^ma_iS_i
$$

$$
T_{mod}=\sum_{i=1}^mb_iT_i
$$

subject to: $\sum_{i=1}^na_i=\sum_{i=1}^nb_i=1$.

where $S_i$ and $T_i$ are the shape and texture of $ith$ face.

We define the morphable model as set of faces $S_{mod}(a)$ and $T_{mod}(b)$, parameteried by the coefficient $a=[a_1,a_2,...,a_m]^T$ and $b=[b_1,b_2,...,b_m]^T$. Arbitrary new faces can be generated by varying parameters $a$, $b$ that controls shape and texture.

Note that $S$ and $T$ are spanned by $m$ samples. We will use PCA to transform this space to a subspace spanned by
the eigen vectors of the covariance matrix:

$$
\begin{cases}
S_{mod}=\overline{S}+\sum_{i=1}^{m-1}\alpha_{i}s_i \\
T_{mod}=\overline{T}+\sum_{i=1}^{m-1}\beta_it_i
\end{cases}
$$

where $\overline{S},\overline{T}$ the mean facial shape and texture of the dataset, and $s_i$ and $t_i$ are the eigenvalues of shape and texture computed from the corresponding covariance matrix. 

$\alpha=[\alpha_1,\alpha_2,...,\alpha_{m-1}]^T$ and $\beta=[\beta_1,\beta_2,...,\beta_{m-1}]^T$ are the coefficient of shapes and textures.

We can model the probability of $\alpha$ and $\beta$ (assuming that $\alpha_i,\alpha_j$, $\beta_i,\beta_j$ are independent for $i\neq j$:

$$
p(\alpha)=\exp{\{-\frac{1}{2}\sum_{i=1}^{m-1}(\frac{\alpha_i}{\sigma_i})^2\}}
$$

with $\sigma_i^2$ are the eigenvalues of shape's covariance matrix. The probability of $p(\beta)$ can be determined similarly.

The above representation of facial model was introduced in 1999, and later, there were an improvement of this model, called **Basel Face Model (BFM)**. The shape of a face is devided into 2 components: identity and expression. The identity of a face is the shape of a face in neutral expression. The facial model then became:

$$
\begin{cases}
S_{mod}=\overline{S}+A_{id}\alpha_{id}+A_{exp}\alpha_{exp} \\
T_{mod}=\overline{T}+B_{t}\beta_{t}
\end{cases}
$$

Where $A_{id},A_{exp},B_t$ are the matrix where each row represents the principle component of identity, expression and texture. $\alpha_{id},\alpha_{exp},\beta_t$ are the coefficient corresponding to each principle component.

Here $\alpha_{id}$ is 99d, $\alpha_{exp}$ is 29d and $\beta_t$ are 99d.

I will introduce a technique to match the morphable model to the landmark named "landmark fitting".

## 2. Fitting Morphable Model to landmark

To project the model to image plane:

$$
S=sPR(\overline{S}+A_{id}\alpha_{id}+A_{exp}\alpha_{exp})+t_{2d}
$$

where $s$ is scaling factor, $R$ is $3\times 3$ rotation matrix, $t_{2d}$ is the translation vector of $x,y$ coordinate. $P$ is a $2\times 3$ matrix:

$$
P=
\begin{bmatrix}
1 &0 &0 \\
0 &1 &0
\end{bmatrix}
$$

In landmark fitting, we must find the parameters: $s,R,t,\alpha_{id},\alpha_{exp}$ minimize the landmark position of rendered image and original image landmark. We define: 

$$
E_{I}=\|I_{mod}-I_{input}\|
$$

where $I_{mod}$ are the vertices rendered by algorithm, and $I_{input}$ are the vertices we're trying to match.

The likelihood of $P(I_{input}|\alpha,\beta)$ is given by:

$$
P(I_{input}|\alpha)=\exp{\{-\frac{1}{2}E_{I}\}}
$$

We need to maximize the posterior:

$$
P(\alpha|I_{input})=\frac{P(I_{input}|\alpha)p(\alpha)}{P(I_{input})}
$$

and

$$
\begin{aligned}
\text{argmax}_{\alpha} P(\alpha|I_{input})&=\text{argmax}_{\alpha}(P(I_{input}|\alpha) P(\alpha))\\
&=\text{argmin}_{\alpha}(\frac{1}{2}E_I+\frac{1}{2}\sum_{i=1}^{99}(\frac{\alpha_{id,i}}{\sigma_{id,i}})^2+\sum_{i=1}^{29}(\frac{\alpha_{exp,i}}{\sigma_{exp,i}})^2)
\end{aligned}
$$

Texture is randomly generated and isn't considered in the fitting process.

### 2.1 Estimating affine matrix

Firstly, we must find the affine matrix $R$, translation vector $t$ and scaling factor $s$. We will use the **Golden Standard Algorithm** to estimate these matrix.

We denote the shape in 3D coordinate:

$$
X=
\begin{bmatrix}
X_1 &X_2 &X_3 &... &X_n \\
Y_1 &Y_2 &Y_3 &... &Y_n \\
Z_1 &Z_2 &Z_3 &... &Z_n
\end{bmatrix}
$$

We also denote the shape in 2D render image coordinate

$$
x=
\begin{bmatrix}
x_1 &x_2 &x_3 &... &x_n \\
y_1 &y_2 &y_3 &... &y_n
\end{bmatrix}
$$

We represents $X$ and $x$ in homogeous coordinate:

$$
X_{homo}=
\begin{bmatrix}
X_1 &X_2 &... &X_n \\
Y_1 &Y_2 &... &Y_n \\
Z_1 &Z_2 &... &Z_n \\
1 &1 &... &1
\end{bmatrix}
$$

$$
x_{homo}=
\begin{bmatrix}
x_1 &x_2 &... &x_n \\
y_1 &y_2 &... &y_n \\
1 &1 &... &1
\end{bmatrix}
$$

We must find affine matrix $P$ that satisfied:

$$
x_{homo}=PX_{homo}
$$

Where $P$ is $3\times 4$ matrix, in fact we find the affine matrix $P_{norm}$ that transform from normalized coordinate of $X_{homo}$ to normalized coordinate of $x_{homo}$:

$$
\begin{cases}
X_{norm}=UX_{homo} \\
x_{norm}=Tx_{homo} \\
x_{norm}=P_{norm}X_{norm} \\
P=T^{-1}P_{norm}U
\end{cases}
$$

Where $U$ is $4\times 4$ matrix, $T$ is $3\times 3$ matrix.

#### 2.1.1.  Normalization

We denote:

$$
X=
\begin{bmatrix}
X_1 &X_2 &... &X_n \\
Y_1 &Y_2 &... &Y_n \\
Z_1 &Z_2 &... &Z_n
\end{bmatrix}
$$

$$
x=
\begin{bmatrix}
x_1 &x_2 &... &x_n \\
y_1 &y_2 &... &y_n
\end{bmatrix}
$$

Hence:

$$
meanX=
\begin{bmatrix}
\overline{X} \\
\overline{Y} \\
\overline{Z}
\end{bmatrix}
$$

$$
meanx=
\begin{bmatrix}
\overline{x} \\
\overline{y}
\end{bmatrix}
$$

$$
normX=\frac{\sum_{i=1}^n\sqrt{(X_i-\overline{X})^2+(Y_i-\overline{Y})^2+(Z_i-\overline{Z})^2}}{\sqrt{3}n}
$$

$$
normx=\frac{\sum_{i=1}^n\sqrt{(x_i-\overline{x})^2+(y_i-\overline{y})^2}}{\sqrt{2}n}
$$

And then:

$$
X_{norm}=normX
\begin{bmatrix}
X_1-\overline{X} &X_2-\overline{X} &... &X_n-\overline{X} \\
Y_1-\overline{Y} &Y_2-\overline{Y} &... &Y_n-\overline{Y} \\
Z_1-\overline{Z} &Z_2-\overline{Z} &... &Z_n-\overline{Z}
\end{bmatrix}
$$

$$
x_{norm}=normx
\begin{bmatrix}
x_1-\overline{x} &x_2-\overline{x} &... &x_n-\overline{x} \\
y_1-\overline{y} &y_2-\overline{y} &... &y_n-\overline{y}
\end{bmatrix}
$$

We compute $T$ matrix that transforms $x_{norm}=Tx$:

$$
T=
\begin{bmatrix}
normx &0 &-normx\overline{x} \\
0 &normx &-normx\overline{y}
\end{bmatrix}
$$

We caculate $U$ similarly:

$$
U=
\begin{bmatrix}
normX &0 &0 &-normX\overline{X} \\
0 &normX &0 &-normX\overline{Y} \\
0 &0 &normX &-normX\overline{Z}
\end{bmatrix}
$$

It's easy to verify: $x_{norm}=Tx$ and $X_{norm}=UX$.

We denote:

$$
x_{norm}=
\begin{bmatrix}
\tilde{x}_1 &\tilde{x}_2 &... &\tilde{x}_n \\
\tilde{y}_1 &\tilde{y}_2 &... &\tilde{y}_n
\end{bmatrix}
$$

$$
X_{norm}=
\begin{bmatrix}
\tilde{X}_1 &\tilde{X}_2 &... &\tilde{X}_n \\
\tilde{Y}_1 &\tilde{Y}_2 &... &\tilde{Y}_n \\
\tilde{Z}_1 &\tilde{Z}_2 &... &\tilde{Z}_n
\end{bmatrix}
$$

$$
P_{norm}=
\begin{bmatrix}
r_{11} &r_{12} &r_{13} &t_x \\
r_{21} &r_{22} &r_{23} &t_y
\end{bmatrix}
$$

We must approximate:

$$
\begin{cases}
\tilde{x}_i\approx r_{11}\tilde{X}_i+r_{12}\tilde{Y}_i+r_{13}\tilde{Z}_i+t_x=f_1(\tilde{X}_i,\tilde{Y}_i,\tilde{Z}_i) \\
\tilde{y}_i\approx r_{21}\tilde{X}_i+r_{22}\tilde{Y}_i+r_{23}\tilde{Z}_i+t_y=f_2(\tilde{X}_i,\tilde{Y}_i,\tilde{Z}_i)
\end{cases}
$$

for all $i=\overline{1,n}$. This is just a Linear Regression Problem, we can easily estimate value of $r$ and $t$.

We define loss function:

$$
L=\frac{1}{2}\sum_{i=1}^n(\tilde{x}_i-f_1(\tilde{X}_i,\tilde{Y}_i,\tilde{Z}_i)^2+(\tilde{y}_i-f_2(\tilde{X}_i,\tilde{Y}_i,\tilde{Z}_i))^2)
$$

We can solve this problem by finding derivatives of $L$. In numpy, the optimization can be solved by applying ``np.lstsq`` function.

See this function in ``python``:

```python
def estimate_affine_matrix(x,X):
    '''
    Using golden standard algorithm to estimate
    the affine transformation from 3D coordinate
    to 2D coord.
    Parameters:
    X: (3,n) matrix, where n represents number of points
    x: (2,n) matrix
    Return P that satisfy:
    x=PX
    '''
    # Normalizing x
    mean=np.mean(x,axis=1,keepdims=True)
    x=x-mean
    avg_norm=np.mean(np.sqrt(np.sum(x**2,axis=0)))
    scale=np.sqrt(2)/avg_norm
    x=scale*x # Divide by standard deviation

    # Constructing T matrix
    T=np.zeros((3,3),dtype=np.float32)
    T[0,0]=scale;T[1,1]=scale
    T[0,2]=-scale*mean[0,0]
    T[1,2]=-scale*mean[1,0]
    T[2,2]=1

    # Normalizing X
    mean=np.mean(X,axis=1,keepdims=True)
    X=X-mean
    avg_norm=np.mean(np.sqrt(np.sum(X**2,axis=0)))
    scale=np.sqrt(3)/avg_norm
    X=scale*X

    # Constructing U matrix
    U=np.zeros((4,4),dtype=np.float32)
    U[0,0]=scale;U[1,1]=scale;U[2,2]=scale
    U[0,3]=-scale*mean[0,0]
    U[1,3]=-scale*mean[1,0]
    U[2,3]=-scale*mean[2,0]
    U[3,3]=1

    num_points=x.shape[1] # number of points
    # Compute P_norm
    A=np.zeros((2*num_points,8),dtype=np.float32)
    A[0:2*num_points:2,0:3]=X.transpose()
    A[0:2*num_points:2,3]=1
    A[1:2*num_points:2,4:7]=X.transpose()
    A[1:2*num_points:2,7]=1
    b=np.reshape(x.T,[2*num_points,1])

    coef,_,_,_=np.linalg.lstsq(A,b)
    P_norm=np.zeros((3,4),dtype=np.float32)
    P_norm[0,0]=coef[0];P_norm[0,1]=coef[1];P_norm[0,2]=coef[2];P_norm[0,3]=coef[3]
    P_norm[1,0]=coef[4];P_norm[1,1]=coef[5];P_norm[1,2]=coef[6];P_norm[1,3]=coef[7]
    P_norm[2,3]=1
    return np.linalg.inv(T).dot(np.dot(P_norm,U))
```

### 2.2 Estimating Euler angle from affine matrix

After getting affine matrix $P$, we must find an 3 Euler angles that forms P. We denote 3 Euler angles as: $\psi$ (rotating around $x$ axis), $\theta$ (rotating around $y$ axis), and $\phi$ (rotating around $z$ axis). The rotation matrix is then determined by:

$$
R=R_zR_yR_x=
\begin{bmatrix}
\cos{\theta}\cos{\phi} &\sin{\psi}\sin{\theta}\cos{\phi}-\cos{\psi}\sin{\phi} &\cos{\psi}\sin{\theta}\cos{\phi}+\sin{\psi}\sin{\phi} \\
\cos{\theta}\sin{\phi} &\sin{\psi}\sin{\theta}\sin{\phi}+\cos{\psi}\cos{\phi} &\cos{\psi}\sin{\theta}\sin{\phi}-\sin{\psi}\cos{\phi} \\
-\sin{\theta} &\sin{\psi}\cos{\theta} &\cos{\psi}\cos{\theta}
\end{bmatrix}
$$

From matrix $P$, which is a $3\times 4$ matrix, we can extract the translation vector (last column of $P$) and rotation matrix ($3\times 3$ dimensional matrix of $P$).

We set:

$$
\hat{R}=
\begin{bmatrix}
r_{11} &r_{12} &r_{13} \\
r_{21} &r_{22} &r_{23} \\
r_{31} &r_{32} &r_{33}
\end{bmatrix}
$$

Set $r_1=[r_{11},r_{12},r_{13}]^T$ and $r_{2}=[r_{21},r_{22},r_{23}]^T$. Then, scaling factor $s$ can be determined as:

$$
s=\frac{\|r_1\|_2+\|r_{2}\|_2}{2}
$$

We note that each row of $R$ has norm of 1, then we must normalize each row of $\hat{R}$ so that they're 1 norm. The third row of $R$ are the cross product of the first 2 row, then $\hat{R}$ can be rewritten as:

$$
\hat{R}=
\begin{bmatrix}
\frac{r_1}{\|r_1\|_2} \\
\frac{r_2}{\|r_2\|_2} \\
\text{cross}(r_1,r_2)
\end{bmatrix}
$$

From this new $\hat{R}$, we can easily obtain $\psi,\theta,\phi$.

$s,R,t$ are retrieve in the ``P2sRt`` function in ``process.py``:

```python
def P2rst(P):
    '''
    Get the translation, rotation matrix and scaling
    factor from matrix P
    Parameters:
    P: an (3,4) matrix
    Return:
    s: scaling factor
    R: rotation matrix
    t: translation
    '''
    t=P[:2,3]
    r1=P[0,:3];r2=P[1,:3]
    scale=(np.linalg.norm(r1)+np.linalg.norm(r2))/2
    
    # Recalculate matrix R
    r1=r1/np.linalg.norm(r1)
    r2=r2/np.linalg.norm(r2)
    r3=np.cross(r1,r2)

    # Forming R
    R=np.zeros((3,3),dtype=np.float32)
    R[0]=r1
    R[1]=r2
    R[2]=r3

    return t,R,scale
```

## 2.3. Landmark fitting

Shape of a face can be represented as:

$$
S=\overline{S}+A_{id}\alpha_{id}+A_{exp}\alpha_{exp}
$$

which is a $3n\times 1$ vector, where $n$ is the number of vertices.

Project the shape to the image plane, we can use:

$$
I_{mod}=sPRS+t_{2d}
$$

In this case, $S$ is reshaped into $3\times n$ matrix, which can be obtained by ``S=np.reshape(S,[n,3]).T``.

The objective function then becomes:

$$
L=\frac{1}{2}\|sPR(\overline{S}+A_{id}\alpha_{id}+A_{exp}\alpha_{exp})+t_{2d}-I\|_2^2+\frac{1}{2}\sum_{i=1}^{99}(\frac{\alpha_{i,id}}{\sigma_{i,id}})^2+\frac{1}{2}\sum_{i=1}^{29}(\frac{\alpha_{i,exp}}{\sigma_{i,exp}})^2
$$

I have mentioned that $\sigma_{i,id}$ are the eigenvalue corresponding to principle component $i$ of the identity covariance matrix, and $\sigam_{i,exp}$ are the eigenvalue correponding to the principle component $i$ of the expression covariance matrix. These values are denoted as ``shapeEV`` and ``expEV`` in the code.

In order to jointly optimize both $\alpha_{id}$ and $\alpha_{exp}$, we fix one and try to optimize other. I will introduce how to optimize $\alpha_{id}$, and $\alpha_{exp}$ can be implemented similarly.

We note that the term $\overline{S}+A_{exp}\alpha_{exp}$ are fixed, and is $3n\times 1$ matrix. We reshape it into $3\times n$ matrix:

```python
u=np.dot(expPC,ep)+shapeMU
b=s*np.dot(P,R.dot(np.reshape(u,[3,-1]).T))
b=np.reshape(b,[-1,1])
```

``b`` will be a fixed term.

We denote:

$$
A_{id}=
\begin{bmatrix}
a_{1,1} &a_{1,2} &... &a_{1,99} \\
a_{2,1} &a_{2,2} &... &a_{2,99} \\
... &... &... &... \\
a_{3n,1} &a_{3n,2} &... &a_{3n,99}
\end{bmatrix}
$$

$$
\alpha_{id}=[\alpha_1,\alpha_2,...,\alpha_{99}]^T
$$

then

$$
A_{id}\alpha_{id}=
\begin{bmatrix}
\sum_{i=1}^{99}a_{1,i}\alpha_i \\
\sum_{i=1}^{99}a_{2,i}\alpha_i \\
... \\
\sum_{i=1}^{99}a_{3n,i}\alpha_i
\end{bmatrix}
$$

We denote:

$$
A=sPR=
\begin{bmatrix}
r_{11} &r_{12} &r_{13} \\
r_{21} &r_{22} &r_{23}
\end{bmatrix}
$$


We reshape $A_{id}\alpha_{id}$ into a $3\times n$ matrix and multiply it to A:

$$
M=
\begin{bmatrix}
r_{11}\sum_{i=1}^{99}a_{1,i}\alpha_i+r_{12}\sum_{i=1}^{99}a_{2,i}\alpha_i+r_{13}\sum_{i=1}^{99}a_{3,i}\alpha_i &...\\
r_{21}\sum_{i=1}^{99}a_{1,i}\alpha_i+r_{22}\sum_{i=2}^{99}a_{2,i}\alpha_i+r_{23}\sum_{i=1}^{99}a_{3,i}\alpha_i &...
\end{bmatrix}
$$

$M$ is a $2\times n$ matrix, where column represent the $x,y$ coordinate of a point. Then we can flatten M by applying ``M=np.reshape(M.T,[-1,1])``, then $M$ becomes:

$$
M=
\begin{bmatrix}
r_{11}\sum_{i=1}^{99}a_{1,i}\alpha_i+r_{12}\sum_{i=1}^{99}a_{2,i}\alpha_i+r_{13}\sum_{i=1}^{99}a_{3,i}\alpha_i \\
r_{21}\sum_{i=1}^{99}a_{1,i}\alpha_i+r_{22}\sum_{i=2}^{99}a_{2,i}\alpha_i+r_{23}\sum_{i=1}^{99}a_{3,i}\alpha_i \\
... \\
r_{11}\sum_{i=1}^{99}a_{3n-2,i}\alpha_i+r_{12}\sum_{i=1}^{99}a_{3n-1,i}\alpha_i+r_{13}\sum_{i=1}^{99}a_{3n,i}\alpha_i \\
r_{21}\sum_{i=1}^{99}a_{3n-2,i}\alpha_i+r_{22}\sum_{i=2}^{99}a_{3n-1,i}\alpha_i+r_{23}\sum_{i=1}^{99}a_{3n,i}\alpha_i
\end{bmatrix}
$$

$$
M=
\begin{bmatrix}
r_{11}a_{1,1}+r_{12}a_{2,1}+r_{13}a_{3,1} &r_{11}a_{1,2}+r_{12}a_{2,2}+r_{13}a_{3,2} &... \\
r_{21}a_{1,1}+r_{22}a_{2,1}+r_{23}a_{3,1} &r_{21}a_{1,2}+r_{22}a_{2,2}+r_{23}a_{3,2} &... \\
... \\
r_{11}a_{3n-2,1}+r_{12}a_{3n-1,1}+r_{13}a_{3n,1} &r_{11}a_{3n-2,2}+r_{12}a_{3n-1,2}+r_{13}a_{3n,2} &... \\
r_{21}a_{3n-2,1}+r_{22}a_{3n-1,1}+r_{23}a_{3n,1} &r_{21}a_{3n-2,2}+r_{22}a_{3n-1,2}+r_{23}a_{3n,2} &...
\end{bmatrix}
\begin{bmatrix}
\alpha_1 \\
\alpha_2 \\
... \\
\alpha_{99}
\end{bmatrix}
$$

We set:

$$
D=
\begin{bmatrix}
r_{11}a_{1,1}+r_{12}a_{2,1}+r_{13}a_{3,1} &r_{11}a_{1,2}+r_{12}a_{2,2}+r_{13}a_{3,2} &... \\
r_{21}a_{1,1}+r_{22}a_{2,1}+r_{23}a_{3,1} &r_{21}a_{1,2}+r_{22}a_{2,2}+r_{23}a_{3,2} &... \\
... \\
r_{11}a_{3n-2,1}+r_{12}a_{3n-1,1}+r_{13}a_{3n,1} &r_{11}a_{3n-2,2}+r_{12}a_{3n-1,2}+r_{13}a_{3n,2} &... \\
r_{21}a_{3n-2,1}+r_{22}a_{3n-1,1}+r_{23}a_{3n,1} &r_{21}a_{3n-2,2}+r_{22}a_{3n-1,2}+r_{23}a_{3n,2} &...
\end{bmatrix}
$$

We can easily obtain $D$ and optimize $\alpha_{id}$. This can be done in the following code:

```python
def estimate_shape_params(lm,shapeMU,shapePC,exp,s,R,t,shapeEV,lambd):
    '''
    Estimating shape parameters
    Parameters:
    lm: landmark, of shape [2n,1]
    shapeMU: mean shape, of shape [3n,1]
    shapePC: identity parameters, of shape [3n,99]
    exp: expression, of shape [3n,1]
    s: scaling factor, a scalar
    R: rotation matrix, of shape [3,3]
    t: translation matrix, of shape [2,1] (translation of x and y coord)
    shapeEV: identity std, for regularization, of shape [99,1]
    lambd: regularization term
    '''
    n_vertices=shapePC.shape[0]//3
    n_feats=shapePC.shape[1]
    t=np.array(t)
    u=shapeMU+exp
    P=np.array([[1,0,0],[0,1,0]],dtype=np.float32)
    A=s*np.dot(P,R)
    b=np.dot(A,np.reshape(u,[-1,3]).T)
    b=np.reshape(b,[-1,1])+np.tile(t[:,np.newaxis],[n_vertices,1])
    U=np.zeros((3,n_vertices*n_feats))
    U[0,:]=shapePC[0:3*n_vertices+1:3,:].flatten()
    U[1,:]=shapePC[1:3*n_vertices+1:3,:].flatten()
    U[2,:]=shapePC[2:3*n_vertices+1:3,:].flatten()
    D=np.reshape(np.dot(R[:2,:],U),[2,n_vertices,n_feats])
    D=np.reshape(np.transpose(D,[1,0,2]),[-1,n_feats])
    # We have the equation
    right_eq=np.dot(D.T,lm-b)
    shapeEV=1/shapeEV.flatten()
    left_eq=np.dot(D.T,D)+lambd*np.diag(shapeEV**2)
    sp=np.linalg.inv(left_eq).dot(right_eq)
    return sp
```

The same can be applied to $\alpha_{exp}$:

```python
def estimate_expression_params(lm,shapeMU,expPC,shape,s,R,t,expEV,lambd):
    '''
    Estimate expression parameters.
    Parameters:
    lm: landmark array position, of shape [2n,1]
    shapeMU: mean shape coefficient, in shape [3n,1]
    expPC: expression parameters, of shape [3n,29]
    shape: shape, of shape [3n,1]
    s: scaling factor, a scalar
    R: rotation matrix, of shape [3,3]
    t: translation matrix, of shape [3,1]
    expEV: expression std, for regularization purpose
    lambd: regularization parameters
    '''
    n_vertices=expPC.shape[0]//3
    n_feats=expPC.shape[1]
    t=np.array(t)
    u=shapeMU+shape
    P=np.array([[1,0,0],[0,1,0]],dtype=np.float32)
    A=s*np.dot(P,R)
    b=np.dot(A,np.reshape(u,[-1,3]).T)
    b=np.reshape(b,[-1,1])+np.tile(t[:,np.newaxis],[n_vertices,1])
    U=np.zeros((3,n_vertices*n_feats))
    U[0,:]=expPC[0:3*n_vertices+1:3,:].flatten()
    U[1,:]=expPC[1:3*n_vertices+1:3,:].flatten()
    U[2,:]=expPC[2:3*n_vertices+1:3,:].flatten()
    D=np.reshape(np.dot(R[:2,:],U),[2,n_vertices,n_feats])
    D=np.reshape(np.transpose(D,[1,0,2]),[-1,n_feats])
    right_eq=np.dot(D.T,lm-b)
    expEV=1/expEV.flatten()
    left_eq=np.dot(D.T,D)+lambd*np.diagflat(expEV**2)
    ep=np.linalg.inv(left_eq).dot(right_eq)
    return ep
```

Finally, we combine and get the landmark fitting algorithm:

```python
def fitting_landmarks(lm,index,bfm,lambd1=20,lambd2=40,n_iters=3):
    '''
    Fitting landmark
    Parameters:
    lm: array of landmark, of shape [128,1]
    index: index of fitting vertices
    bfm: MorphableModel object
    lambd1: regularizaton term for expression
    lambd2: regularization term for shape
    n_iters: number of iteration
    '''
    # Initialize sp and ep
    sp=np.zeros((bfm.n_shape_para,1))
    ep=np.zeros((bfm.n_exp_para,1))
    # Get the position for each vertice (including x,y,z position)
    index=np.tile(index[:,np.newaxis],[1,3])*3
    index[:,1]+=1
    index[:,2]+=2
    index_all=index.flatten()
    
    shapeMU=bfm.model['shapeMU'][index_all,:1]
    shapePC=bfm.model['shapePC'][index_all,:bfm.n_shape_para]
    expPC=bfm.model['expPC'][index_all,:bfm.n_exp_para]

    s=4e-04
    R=estimate_rotation_matrix(angles=[0,0,0])
    t=[0,0,0]
    expEV=bfm.model['expEV']
    shapeEV=bfm.model['shapeEV']

    for i in range(n_iters):
        print('Iteration {}:'.format(i))
        X=shapeMU+np.dot(expPC,ep)+np.dot(shapePC,sp)
        P=estimate_affine_matrix(np.reshape(lm,[-1,2]).T,np.reshape(X,[-1,3]).T)
        t,R,s=P2rst(P)

        # fitting
        shape=np.dot(shapePC,sp)
        ep=estimate_expression_params(lm,shapeMU,expPC,shape,s,R,t,expEV,lambd1)
        exp=np.dot(expPC,ep)
        sp=estimate_shape_params(lm,shapeMU,shapePC,exp,s,R,t,shapeEV,lambd2)

    return s,R,t,sp,ep
```